In [58]:
from barlowbert_dm import DataCollatorForBarlowBertWithMLM
from barlowbert_models import BarlowBert
from transformers import BertTokenizerFast, CONFIG_MAPPING, BertModel
from datasets import load_dataset, load_from_disk, concatenate_datasets
import torch
from tqdm import tqdm
from typing import Dict
import pdb

In [2]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', padding=True,truncation=True,)
collator = DataCollatorForBarlowBertWithMLM(tokenizer,mlm_probability=0.2)

In [3]:
data = load_from_disk('/mounts/data/proj/jabbar/barlowbert/bookcorpus_20mil_128')

In [4]:
data.set_format(type='torch', columns=['input_ids', 'attention_mask'])

In [5]:
train_loader = torch.utils.data.DataLoader(data, batch_size=4, num_workers=2,
                                            pin_memory=True, collate_fn=collator)

In [6]:
(x,y) = next(iter(train_loader))

In [7]:
bert_tiny = {
    "hidden_size" : 128 ,
    "num_hidden_layers" : 2,
    "num_attention_heads": int(128/64),
    "intermediate_size" : int(128*4)
}

In [8]:
config = CONFIG_MAPPING['bert']()
config.update(bert_tiny)
config.projector='128-128'

In [22]:
bb_model = BarlowBert(config)

In [23]:
out = bb_model(**y)

> /mounts/Users/cisintern/jabbar/git/barlowtwins/pl_model/barlowbert_models.py(92)forward()
     90 
     91         return_dict = return_dict if return_dict is not None else self.config.use_return_dict
---> 92 
     93         outputs = self.bert(
     94             input_ids,



ipdb>  q


BdbQuit: 

In [81]:
class Pooling(torch.nn.Module):
    """Performs pooling (max or mean) on the token embeddings.
    Using pooling, it generates from a variable sized sentence a fixed sized sentence embedding. This layer also allows to use the CLS token if it is returned by the underlying word embedding model.
    You can concatenate multiple poolings together.
    :param word_embedding_dimension: Dimensions for the word embeddings
    :param pooling_mode: Can be a string: mean/max/cls. If set, overwrites the other pooling_mode_* settings
    :param pooling_mode_cls_token: Use the first token (CLS token) as text representations
    :param pooling_mode_max_tokens: Use max in each dimension over all tokens.
    :param pooling_mode_mean_tokens: Perform mean-pooling
    :param pooling_mode_mean_sqrt_len_tokens: Perform mean-pooling, but devide by sqrt(input_length).
    """
    def __init__(self,
                 word_embedding_dimension: int,
                 pooling_mode: str = None,
                 pooling_mode_cls_token: bool = False,
                 pooling_mode_max_tokens: bool = False,
                 pooling_mode_mean_tokens: bool = True,
                 pooling_mode_mean_sqrt_len_tokens: bool = False,
                 ):
        super(Pooling, self).__init__()

        self.config_keys = ['word_embedding_dimension',  'pooling_mode_cls_token', 'pooling_mode_mean_tokens', 'pooling_mode_max_tokens', 'pooling_mode_mean_sqrt_len_tokens']

        if pooling_mode is not None:        #Set pooling mode by string
            pooling_mode = pooling_mode.lower()
            assert pooling_mode in ['mean', 'max', 'cls']
            pooling_mode_cls_token = (pooling_mode == 'cls')
            pooling_mode_max_tokens = (pooling_mode == 'max')
            pooling_mode_mean_tokens = (pooling_mode == 'mean')

        self.word_embedding_dimension = word_embedding_dimension
        self.pooling_mode_cls_token = pooling_mode_cls_token
        self.pooling_mode_mean_tokens = pooling_mode_mean_tokens
        self.pooling_mode_max_tokens = pooling_mode_max_tokens
        self.pooling_mode_mean_sqrt_len_tokens = pooling_mode_mean_sqrt_len_tokens

        pooling_mode_multiplier = sum([pooling_mode_cls_token, pooling_mode_max_tokens, pooling_mode_mean_tokens, pooling_mode_mean_sqrt_len_tokens])
        self.pooling_output_dimension = (pooling_mode_multiplier * word_embedding_dimension)


    def __repr__(self):
        return "Pooling({})".format(self.get_config_dict())

    def get_pooling_mode_str(self) -> str:
        """
        Returns the pooling mode as string
        """
        modes = []
        if self.pooling_mode_cls_token:
            modes.append('cls')
        if self.pooling_mode_mean_tokens:
            modes.append('mean')
        if self.pooling_mode_max_tokens:
            modes.append('max')
        if self.pooling_mode_mean_sqrt_len_tokens:
            modes.append('mean_sqrt_len_tokens')

        return "+".join(modes)

    def forward(self, bert_encoder_output,attention_mask):
        pdb.set_trace()
        token_embeddings = bert_encoder_output.last_hidden_state
        cls_token = bert_encoder_output.last_hidden_state[:,0]

        ## Pooling strategy
        output_vectors = []
        if self.pooling_mode_cls_token:
            output_vectors.append(cls_token)
        if self.pooling_mode_max_tokens:
            input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
            token_embeddings[input_mask_expanded == 0] = -1e9  # Set padding tokens to large negative value
            max_over_time = torch.max(token_embeddings, 1)[0]
            output_vectors.append(max_over_time)
        if self.pooling_mode_mean_tokens or self.pooling_mode_mean_sqrt_len_tokens:
            input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
            sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)

            #If tokens are weighted (by WordWeights layer), feature 'token_weights_sum' will be present
#             if 'token_weights_sum' in features:
#                 sum_mask = features['token_weights_sum'].unsqueeze(-1).expand(sum_embeddings.size())
#             else:
            sum_mask = input_mask_expanded.sum(1)

            sum_mask = torch.clamp(sum_mask, min=1e-9)

            if self.pooling_mode_mean_tokens:
                output_vectors.append(sum_embeddings / sum_mask)
            if self.pooling_mode_mean_sqrt_len_tokens:
                output_vectors.append(sum_embeddings / torch.sqrt(sum_mask))

        output_vector = torch.cat(output_vectors, 1)
#         features.update({'sentence_embedding': output_vector})
        return output_vector

    def get_sentence_embedding_dimension(self):
        return self.pooling_output_dimension

    def get_config_dict(self):
        return {key: self.__dict__[key] for key in self.config_keys}

    def save(self, output_path):
        with open(os.path.join(output_path, 'config.json'), 'w') as fOut:
            json.dump(self.get_config_dict(), fOut, indent=2)

    @staticmethod
    def load(input_path):
        with open(os.path.join(input_path, 'config.json')) as fIn:
            config = json.load(fIn)

        return Pooling(**config)

In [82]:
config.mean_pooling = False
config.max_pooling = False
config.cls_pooling = True

In [83]:
config.output_attentions = True

In [84]:
pooler = Pooling(128)

In [85]:
model = BertModel(config)

In [86]:
enc = model(**y)

In [88]:
pool = pooler(enc,y)

> <ipython-input-81-854b509078dd>(62)forward()
     60     def forward(self, bert_encoder_output,inp):
     61         pdb.set_trace()
---> 62         token_embeddings = bert_encoder_output.last_hidden_state
     63         attention_mask = inp['attention_mask']
     64         cls_token = bert_encoder_output.last_hidden_state[:,0]



ipdb>  c


In [89]:
pool.shape

torch.Size([4, 128])

In [90]:
config

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "cls_pooling": true,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_pooling": false,
  "max_position_embeddings": 512,
  "mean_pooling": false,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "output_attentions": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "projector": "128-128",
  "transformers_version": "4.8.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [44]:
enc.attentions[-1].unsqueeze(-1).expand_like(token_embeddings)

AttributeError: 'Tensor' object has no attribute 'expand_like'